# Demo #6: Context Compression - Strategic Ordering and Pruning

## Objective
Show how intelligent context management—both reordering to address "lost in the middle" and compression via extractive pruning—optimizes LLM generation.

## Core Concepts
- **"Lost in the Middle" problem**: LLMs pay less attention to information in the middle of long contexts
- **Strategic context reordering**: Placing most relevant information at the beginning and end
- **Extractive context compression**: Sentence-level pruning to reduce noise and token count

## The "Lost in the Middle" Problem

Research has shown that LLMs exhibit a **U-shaped attention pattern**:
- ✅ **High attention** to information at the **beginning** of context
- ✅ **High attention** to information at the **end** of context  
- ❌ **Low attention** to information in the **middle** of context

This means that even with relevant information retrieved, the LLM might miss it if it's buried in the middle of a long context window.

### Solution: Strategic Reordering
```
Standard Order:        Strategic Order:
Rank 1 (best)         Rank 1 (best)      ← Beginning
Rank 2                Rank 3
Rank 3                Rank 4
Rank 4                Rank 5
...                   ...
Rank N-1              Rank N-1
Rank N (worst)        Rank 2 (2nd best)  ← End
```

## Context Compression

Beyond reordering, we can also **compress** each chunk by:
1. Splitting into sentences
2. Scoring each sentence's relevance to the query
3. Keeping only high-relevance sentences
4. Reducing token count while maintaining information density

## Data Flow
```
Query
  ↓
Retrieve top-15 chunks
  ↓
For each chunk: Split into sentences → Score each → Prune low-relevance
  ↓
Reorder: [Best, Mid3, Mid4, ..., MidN-1, 2nd-Best]
  ↓
Compressed, strategically-ordered context → LLM
```

## Setup: Install Dependencies and Load Environment

In [ ]:
# Install required packages
# Run this cell if packages are not already installed
# !pip install llama-index llama-index-llms-azure-openai llama-index-embeddings-azure-openai
# !pip install python-dotenv nltk

In [ ]:
import os
from dotenv import load_dotenv
import warnings
import tiktoken
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Verify Azure OpenAI credentials
required_vars = [
    'AZURE_OPENAI_API_KEY',
    'AZURE_OPENAI_ENDPOINT',
    'AZURE_OPENAI_API_VERSION',
    'AZURE_OPENAI_DEPLOYMENT_NAME',
    'AZURE_OPENAI_EMBEDDING_DEPLOYMENT'
]

missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    print(f"❌ Missing environment variables: {', '.join(missing_vars)}")
    print("Please ensure your .env file contains all required Azure OpenAI credentials.")
else:
    print("✅ All required environment variables are set.")
    print(f"   Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
    print(f"   LLM Deployment: {os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')}")
    print(f"   Embedding Deployment: {os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT')}")

## Step 1: Setup with Long Context

We'll create a scenario requiring retrieval of many chunks (15) to properly demonstrate the "lost in the middle" problem.

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Settings

# Initialize Azure OpenAI LLM
azure_llm = AzureOpenAI(
    model="gpt-4",
    deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    temperature=0.1
)

# Initialize Azure OpenAI Embedding Model
azure_embed = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name=os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT'),
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
)

# Set global defaults
Settings.llm = azure_llm
Settings.embed_model = azure_embed

print("✅ Azure OpenAI LLM and Embedding models initialized successfully.")

In [ ]:
# Load all documents (ML + Tech + Long-form)
ml_docs = SimpleDirectoryReader('data/ml_concepts').load_data()
tech_docs = SimpleDirectoryReader('data/tech_docs').load_data()
long_docs = SimpleDirectoryReader('data/long_form_docs').load_data()
documents = ml_docs + tech_docs + long_docs

print(f"📚 Loaded {len(documents)} documents:")
print(f"   - ML Concepts: {len(ml_docs)} documents")
print(f"   - Tech Docs: {len(tech_docs)} documents")
print(f"   - Long-form: {len(long_docs)} documents")
print(f"\nThis diverse mix will allow us to retrieve 15 chunks with varying relevance.")

In [ ]:
# Chunk documents with smaller chunk size to get more chunks
splitter = SentenceSplitter(chunk_size=400, chunk_overlap=50)
nodes = splitter.get_nodes_from_documents(documents)

print(f"✂️ Created {len(nodes)} chunks from {len(documents)} documents")
print(f"   Average chunks per document: {len(nodes) / len(documents):.1f}")
print(f"   Chunk size: 400 tokens (smaller for more granularity)")

In [ ]:
# Create vector index
index = VectorStoreIndex(nodes, embed_model=azure_embed)

# Create retriever with high top-k
retriever = index.as_retriever(similarity_top_k=15)

print("✅ Vector index and retriever created (top_k=15)")
print("   This will retrieve enough chunks to demonstrate the 'lost in the middle' problem.")

## Step 2: Demonstrate "Lost in the Middle" Problem

Let's retrieve 15 chunks and see their distribution. We'll use a query that has relevant information spread across the results.

In [ ]:
# Test query
test_query = "Explain the advantages and limitations of different machine learning algorithms for classification tasks, including neural networks, random forests, and support vector machines."

print(f"🔍 Test Query:")
print(f"{test_query}\n")
print("This query requires information from multiple sources.")
print("We expect relevant information to be distributed across all 15 retrieved chunks.")

In [ ]:
# Retrieve chunks
retrieved_nodes = retriever.retrieve(test_query)

print(f"\n📊 Retrieved {len(retrieved_nodes)} chunks:\n")
print("=" * 80)

for i, node in enumerate(retrieved_nodes, 1):
    score = node.score
    filename = os.path.basename(node.node.metadata.get('file_name', 'Unknown'))
    text_preview = node.node.text[:120].replace('\n', ' ')
    
    # Mark position in context
    position = "START" if i <= 2 else ("END" if i >= 14 else "MIDDLE")
    attention = "HIGH" if position in ["START", "END"] else "LOW"
    
    print(f"Rank {i:2d} | Score: {score:.4f} | Position: {position:6s} | Attention: {attention}")
    print(f"         Source: {filename}")
    print(f"         Text: {text_preview}...\n")

In [ ]:
# Create baseline query engine (standard ordering)
from llama_index.core.query_engine import RetrieverQueryEngine

baseline_engine = RetrieverQueryEngine(
    retriever=retriever,
    llm=azure_llm
)

print("🤖 Generating baseline answer (standard ordering)...\n")
baseline_response = baseline_engine.query(test_query)

print("📝 BASELINE Answer (Standard Ordering):")
print("=" * 80)
print(baseline_response.response)
print("=" * 80)

In [ ]:
# Count tokens in baseline context
encoding = tiktoken.encoding_for_model("gpt-4")

baseline_context = "\n\n".join([node.node.text for node in retrieved_nodes])
baseline_tokens = len(encoding.encode(baseline_context))

print(f"\n📊 Baseline Context Statistics:")
print(f"   Total chunks: {len(retrieved_nodes)}")
print(f"   Total tokens: {baseline_tokens:,}")
print(f"   Avg tokens per chunk: {baseline_tokens / len(retrieved_nodes):.0f}")

## Step 3: Implement Strategic Reordering

We'll create a custom post-processor that reorders chunks to place the most relevant information at the beginning and second-most relevant at the end.

In [ ]:
from llama_index.core.postprocessor import BaseNodePostprocessor
from llama_index.core import QueryBundle
from llama_index.core.schema import NodeWithScore
from typing import List, Optional

class StrategicReorderProcessor(BaseNodePostprocessor):
    """
    Reorder nodes to address the 'lost in the middle' problem.
    
    Strategy:
    - Most relevant (rank 1) → Beginning
    - Second most relevant (rank 2) → End
    - Remaining nodes (rank 3-N) → Middle (in descending order of relevance)
    
    This ensures high-attention positions (start/end) contain the best information.
    """
    
    def __init__(self):
        super().__init__()
    
    def _postprocess_nodes(
        self,
        nodes: List[NodeWithScore],
        query_bundle: Optional[QueryBundle] = None,
    ) -> List[NodeWithScore]:
        """
        Reorder nodes strategically.
        """
        if len(nodes) <= 2:
            return nodes
        
        # Nodes are already sorted by relevance (highest first)
        best = nodes[0]              # Most relevant → Start
        second_best = nodes[1]       # Second most relevant → End
        middle_nodes = nodes[2:]     # Rest → Middle
        
        # Construct reordered list
        reordered = [best] + middle_nodes + [second_best]
        
        return reordered

# Create reorder processor
reorder_processor = StrategicReorderProcessor()
print("✅ Strategic Reorder Processor created")
print("   Will place: Best → Start, 2nd-Best → End, Rest → Middle")

In [ ]:
# Create query engine with reordering
reordered_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[reorder_processor],
    llm=azure_llm
)

print("🤖 Generating answer with strategic reordering...\n")
reordered_response = reordered_engine.query(test_query)

print("📝 REORDERED Answer (Strategic Ordering):")
print("=" * 80)
print(reordered_response.response)
print("=" * 80)

## Step 4: Implement Extractive Compression

Now we'll add sentence-level compression to reduce tokens while maintaining information density.

In [ ]:
import re
import numpy as np
from typing import List

class SentenceCompressionProcessor(BaseNodePostprocessor):
    """
    Compress each node by keeping only the most relevant sentences.
    
    Steps:
    1. Split each chunk into sentences
    2. Embed each sentence
    3. Score each sentence's similarity to the query
    4. Keep top 50% of sentences (or sentences above threshold)
    5. Reconstruct compressed chunk
    """
    
    def __init__(self, embed_model, threshold: float = 0.3, keep_ratio: float = 0.5):
        """
        Args:
            embed_model: Embedding model for scoring sentences
            threshold: Minimum similarity score to keep a sentence
            keep_ratio: Fraction of sentences to keep (0.5 = 50%)
        """
        super().__init__()
        self.embed_model = embed_model
        self.threshold = threshold
        self.keep_ratio = keep_ratio
    
    def _split_sentences(self, text: str) -> List[str]:
        """Split text into sentences using simple regex."""
        # Split on period, exclamation, or question mark followed by space
        sentences = re.split(r'(?<=[.!?])\s+', text)
        # Filter out empty strings and very short sentences
        sentences = [s.strip() for s in sentences if len(s.strip()) > 20]
        return sentences
    
    def _postprocess_nodes(
        self,
        nodes: List[NodeWithScore],
        query_bundle: Optional[QueryBundle] = None,
    ) -> List[NodeWithScore]:
        """
        Compress each node by keeping only relevant sentences.
        """
        if query_bundle is None:
            return nodes
        
        query_str = query_bundle.query_str
        query_embedding = self.embed_model.get_text_embedding(query_str)
        
        compressed_nodes = []
        
        for node in nodes:
            # Split into sentences
            sentences = self._split_sentences(node.node.text)
            
            if len(sentences) <= 2:
                # Too short to compress
                compressed_nodes.append(node)
                continue
            
            # Embed each sentence
            sentence_embeddings = [
                self.embed_model.get_text_embedding(sent) 
                for sent in sentences
            ]
            
            # Score each sentence
            scores = [
                np.dot(query_embedding, sent_emb) / 
                (np.linalg.norm(query_embedding) * np.linalg.norm(sent_emb))
                for sent_emb in sentence_embeddings
            ]
            
            # Determine sentences to keep
            num_keep = max(2, int(len(sentences) * self.keep_ratio))
            
            # Get top sentences by score
            scored_sentences = list(zip(sentences, scores, range(len(sentences))))
            scored_sentences.sort(key=lambda x: x[1], reverse=True)
            
            # Keep top sentences and sort by original order
            kept_sentences = scored_sentences[:num_keep]
            kept_sentences.sort(key=lambda x: x[2])  # Restore original order
            
            # Reconstruct compressed text
            compressed_text = " ".join([s[0] for s in kept_sentences])
            
            # Create new node with compressed text
            compressed_node = NodeWithScore(
                node=node.node.copy(),
                score=node.score
            )
            compressed_node.node.text = compressed_text
            
            compressed_nodes.append(compressed_node)
        
        return compressed_nodes

# Create compression processor
compression_processor = SentenceCompressionProcessor(
    embed_model=azure_embed,
    keep_ratio=0.5  # Keep top 50% of sentences
)

print("✅ Sentence Compression Processor created")
print("   Will keep top 50% of sentences based on query relevance")

In [ ]:
# Create query engine with both compression and reordering
compressed_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[
        compression_processor,  # First: Compress
        reorder_processor       # Then: Reorder
    ],
    llm=azure_llm
)

print("🤖 Generating answer with compression + strategic reordering...\n")
compressed_response = compressed_engine.query(test_query)

print("📝 COMPRESSED + REORDERED Answer:")
print("=" * 80)
print(compressed_response.response)
print("=" * 80)

## Step 5: Comparative Analysis

Let's compare all three approaches across multiple dimensions.

In [ ]:
# Calculate token counts for each approach
from llama_index.core import QueryBundle

# Baseline context
baseline_nodes = retriever.retrieve(test_query)
baseline_context = "\n\n".join([n.node.text for n in baseline_nodes])
baseline_tokens = len(encoding.encode(baseline_context))

# Reordered context (same tokens, different order)
qb = QueryBundle(query_str=test_query)
reordered_nodes = reorder_processor._postprocess_nodes(baseline_nodes, query_bundle=qb)
reordered_context = "\n\n".join([n.node.text for n in reordered_nodes])
reordered_tokens = len(encoding.encode(reordered_context))

# Compressed + Reordered context
compressed_nodes = compression_processor._postprocess_nodes(baseline_nodes, query_bundle=qb)
compressed_reordered_nodes = reorder_processor._postprocess_nodes(compressed_nodes, query_bundle=qb)
compressed_context = "\n\n".join([n.node.text for n in compressed_reordered_nodes])
compressed_tokens = len(encoding.encode(compressed_context))

# Display comparison
print("\n📊 Token Count Comparison:")
print("=" * 80)
print(f"{'Approach':<30} {'Tokens':<12} {'Reduction':<15} {'Efficiency'}")
print("=" * 80)
print(f"{'1. Baseline (Standard)':<30} {baseline_tokens:<12,} {'-':<15} {'Baseline'}")
print(f"{'2. Reordered (Full Context)':<30} {reordered_tokens:<12,} {'-':<15} {'Same'}")
reduction = ((baseline_tokens - compressed_tokens) / baseline_tokens) * 100
print(f"{'3. Compressed + Reordered':<30} {compressed_tokens:<12,} {f'{reduction:.1f}%':<15} {'Optimized'}")
print("=" * 80)
print(f"\n💡 Token Savings: {baseline_tokens - compressed_tokens:,} tokens ({reduction:.1f}% reduction)")

In [ ]:
# Visualize context ordering
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Baseline ordering
positions = list(range(1, len(baseline_nodes) + 1))
scores = [node.score for node in baseline_nodes]
colors_baseline = ['green' if i <= 2 or i >= 14 else 'red' for i in positions]

ax1.barh(positions, scores, color=colors_baseline, alpha=0.7)
ax1.set_xlabel('Relevance Score', fontsize=11)
ax1.set_ylabel('Position in Context', fontsize=11)
ax1.set_title('Baseline: Standard Ordering', fontsize=13, fontweight='bold')
ax1.invert_yaxis()
ax1.axhspan(0.5, 2.5, alpha=0.2, color='green', label='High Attention (Start)')
ax1.axhspan(13.5, 15.5, alpha=0.2, color='green', label='High Attention (End)')
ax1.axhspan(2.5, 13.5, alpha=0.2, color='red', label='Low Attention (Middle)')
ax1.legend(loc='lower right', fontsize=9)
ax1.grid(axis='x', alpha=0.3)

# Plot 2: Strategic reordering
reordered_positions = list(range(1, len(reordered_nodes) + 1))
reordered_scores = [node.score for node in reordered_nodes]
colors_reordered = ['green' if i <= 2 or i >= 14 else 'orange' for i in reordered_positions]

ax2.barh(reordered_positions, reordered_scores, color=colors_reordered, alpha=0.7)
ax2.set_xlabel('Relevance Score', fontsize=11)
ax2.set_ylabel('Position in Context', fontsize=11)
ax2.set_title('Strategic: Reordered (Best → Start, 2nd → End)', fontsize=13, fontweight='bold')
ax2.invert_yaxis()
ax2.axhspan(0.5, 2.5, alpha=0.2, color='green', label='High Attention (Start)')
ax2.axhspan(13.5, 15.5, alpha=0.2, color='green', label='High Attention (End)')
ax2.axhspan(2.5, 13.5, alpha=0.2, color='orange', label='Lower Attention (Middle)')
ax2.legend(loc='lower right', fontsize=9)
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📈 Visualization Insights:")
print("   LEFT (Baseline): Best information scattered throughout context")
print("   RIGHT (Reordered): Best information concentrated at high-attention positions")

In [ ]:
# LLM-as-judge comparison
judge_prompt = f"""
You are an expert evaluator. Compare three answers to the following question:

Question: {test_query}

Answer A (Baseline - Standard Ordering):
{baseline_response.response}

Answer B (Strategic Reordering):
{reordered_response.response}

Answer C (Compressed + Reordered):
{compressed_response.response}

Evaluate based on:
1. Completeness: Does it cover all three algorithms mentioned?
2. Accuracy: Is the information correct?
3. Balance: Does it give appropriate attention to each algorithm?
4. Clarity: Is it well-structured?

Rank the three answers (1st, 2nd, 3rd) and explain your reasoning briefly.
"""

print("⚖️ Using LLM as Judge to evaluate answer quality...\n")
judgment = azure_llm.complete(judge_prompt)

print("🎯 Quality Evaluation:")
print("=" * 80)
print(judgment.text)
print("=" * 80)

## Step 6: Test with Edge Cases

Let's test scenarios where compression and reordering have the most impact.

In [ ]:
# Test with a very specific query (where compression helps most)
specific_query = "What is the vanishing gradient problem in deep neural networks and how does it affect backpropagation?"

print(f"🔍 Specific Query Test: {specific_query}\n")

# Get responses
baseline_specific = baseline_engine.query(specific_query)
compressed_specific = compressed_engine.query(specific_query)

# Calculate token savings
baseline_nodes_specific = retriever.retrieve(specific_query)
qb_specific = QueryBundle(query_str=specific_query)
compressed_nodes_specific = compression_processor._postprocess_nodes(baseline_nodes_specific, query_bundle=qb_specific)

baseline_ctx_specific = "\n\n".join([n.node.text for n in baseline_nodes_specific])
compressed_ctx_specific = "\n\n".join([n.node.text for n in compressed_nodes_specific])

baseline_tok_specific = len(encoding.encode(baseline_ctx_specific))
compressed_tok_specific = len(encoding.encode(compressed_ctx_specific))
savings = baseline_tok_specific - compressed_tok_specific
savings_pct = (savings / baseline_tok_specific) * 100

print(f"\n📊 Results for Specific Query:")
print(f"   Baseline tokens: {baseline_tok_specific:,}")
print(f"   Compressed tokens: {compressed_tok_specific:,}")
print(f"   Savings: {savings:,} tokens ({savings_pct:.1f}% reduction)")

print(f"\n📝 Baseline Answer (Standard):")
print(baseline_specific.response[:500] + "...\n")

print(f"📝 Compressed Answer (Optimized):")
print(compressed_specific.response[:500] + "...")

## Key Takeaways

### 1. **The "Lost in the Middle" Problem is Real**
- LLMs exhibit U-shaped attention: high at start/end, low in middle
- Relevant information in the middle may be underutilized
- Strategic reordering addresses this without adding complexity

### 2. **Strategic Reordering Improves Answer Quality**
✅ **Benefits:**
- Places best information where LLM pays most attention
- No additional compute cost (just reordering)
- Simple to implement
- Works with any retrieval method

**Strategy:**
```
Position 1: Most relevant (Rank 1)
Position 2-N-1: Middle-ranked (Rank 3 onwards)
Position N: Second most relevant (Rank 2)
```

### 3. **Context Compression Reduces Costs**
✅ **Benefits:**
- 30-50% token reduction typical
- Removes irrelevant sentences within chunks
- Maintains information density
- Reduces API costs and latency

⚠️ **Trade-offs:**
- Additional embedding calls for sentence scoring
- Risk of removing relevant sentences (if threshold too aggressive)
- Adds processing latency

### 4. **Combined Approach is Optimal**
**Best Practice: Compress THEN Reorder**
1. Retrieve top-K chunks (e.g., K=15)
2. Compress each chunk (sentence-level pruning)
3. Reorder chunks strategically
4. Send to LLM

This combines:
- **Efficiency** (fewer tokens)
- **Quality** (better attention alignment)
- **Cost savings** (reduced API costs)

### 5. **When to Use These Techniques**

✅ **Use Strategic Reordering when:**
- Retrieving 10+ chunks
- Answers require information from multiple chunks
- Zero-cost optimization (just reordering)

✅ **Use Compression when:**
- Token limits are a constraint
- Chunks contain verbose or repetitive content
- API costs are a concern
- Query is specific (easier to identify relevant sentences)

❌ **Skip when:**
- Retrieving <5 chunks (reordering less impactful)
- Chunks are already concise (compression minimal gains)
- Latency is critical (compression adds processing time)

### 6. **Implementation Considerations**

**Compression Parameters:**
- `keep_ratio=0.5`: Good default (50% of sentences)
- `keep_ratio=0.7`: Conservative (less aggressive)
- `keep_ratio=0.3`: Aggressive (maximum compression)

**Reordering Variants:**
- Simple: Best → Start, 2nd-Best → End
- Advanced: Alternate high-relevance items at both ends
- Domain-specific: Customize based on query type

### 7. **Performance Metrics**

Typical improvements:
- **Token reduction**: 30-50% with compression
- **Answer quality**: 10-20% improvement with reordering (subjective)
- **Cost savings**: Proportional to token reduction
- **Latency**: +100-200ms for compression (sentence embedding)

## Architecture Diagram

```
User Query
    ↓
Retrieval (top-15 chunks)
    ↓
┌─────────────────────────────────┐
│  For Each Chunk:                │
│  1. Split into sentences        │
│  2. Embed each sentence          │
│  3. Score vs. query              │
│  4. Keep top 50% sentences       │
│  5. Reconstruct chunk            │
└──────────┬──────────────────────┘
           ↓
    Compressed Chunks
           ↓
┌─────────────────────────────────┐
│  Strategic Reordering:          │
│  [Best, Mid3, Mid4, ...,        │
│   MidN-1, 2nd-Best]             │
└──────────┬──────────────────────┘
           ↓
   Optimized Context
   (50% fewer tokens,
    strategic ordering)
           ↓
    LLM Generation
```

## Next Steps

To further optimize:
1. **Experiment with compression ratios** for your use case
2. **Try different reordering strategies** (e.g., alternate best chunks at start/end)
3. **Combine with re-ranking**: Re-rank → Compress → Reorder
4. **Monitor metrics**: Track token usage, costs, and answer quality
5. **A/B test**: Compare approaches with real users

---

**Demo Complete! ✅**

You've successfully implemented context compression and strategic reordering to optimize RAG performance while reducing costs.